In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('match_data.csv')
df.rename(columns={'Pitch_x': 'x', 'Pitch_y' : 'y',
                   'participation_id' : 'id' , 'Time (s)': 
                   'time', 'Speed (m/s)': 'speed'}, inplace=True)

### Leaderboard Generation
Create a leaderboard ranking athletes based on the three key metrics:
1. Total Distance – The total distance covered by each athlete.
2. Distance at Speed Zone 5 – The cumulative distance an athlete runs while moving at a speed
between 19.8 km/h and 25.1 km/h.
3. Top Speed – The highest speed recorded for each athlete.

In [ ]:
df.sort_values(by=['id', 'time'], inplace=True)

                                              x          y
id                                                        
019d8fa9-3cbd-42f2-bfcf-1bc676d13e0f -18.130694   1.660379
019d8fa9-3cbd-42f2-bfcf-1bc676d13e0f -18.133425   1.645227
019d8fa9-3cbd-42f2-bfcf-1bc676d13e0f -18.136155   1.630076
019d8fa9-3cbd-42f2-bfcf-1bc676d13e0f -18.136155   1.630076
019d8fa9-3cbd-42f2-bfcf-1bc676d13e0f -18.136155   1.630076
...                                         ...        ...
d56ca4de-1be4-499a-8242-b219e051f33c  26.924231 -17.270563
d56ca4de-1be4-499a-8242-b219e051f33c  27.382061 -17.219883
d56ca4de-1be4-499a-8242-b219e051f33c  27.398311 -17.129853
d56ca4de-1be4-499a-8242-b219e051f33c  27.414560 -17.039822
d56ca4de-1be4-499a-8242-b219e051f33c  27.430810 -16.949792

[985274 rows x 2 columns]
